# Multi-Class Prediction of Cirrhosis Outcomes

https://www.kaggle.com/competitions/playground-series-s3e26/data?select=train.csv

## 0 - IMPORTS AND FUNCTIONS

In [ ]:
# Data Analysis and Data Manipulation Libraries
import pandas                  as pd
import numpy                   as np
import seaborn                 as sns
import scikitplot              as skplt
import matplotlib.pyplot       as plt
import plotly.express          as px
import inflection
from ydata_profiling           import ProfileReport 

# Machine Learning Models
import optuna
from sklearn                   import ensemble        as en
from sklearn.linear_model      import LogisticRegression
from catboost                  import CatBoostClassifier
from sklearn.neighbors         import KNeighborsClassifier
import lightgbm                as lgbm

from sklearn                   import model_selection as ms
from sklearn                   import preprocessing   as pp
from sklearn.model_selection   import  KFold, StratifiedKFold

# Metrics and Performance
from sklearn import metrics as mt

# Data Transformer
#from feature_engine.encoding       import CountFrequencyEncoder

# Dysplay
%matplotlib inline
from IPython.core.display    import HTML
from IPython.display         import Image

import warnings
warnings.filterwarnings( 'ignore' )

### 0.1 - HELPER FUNCTIONS

In [ ]:
le = pp.LabelEncoder()
ss = pp.StandardScaler()
rr = pp.RobustScaler()

In [ ]:
def data_trans(df_trans):
    # CATEGORICAL VARIABLES
    df_trans['drug']         = le.fit_transform( df_trans[['drug']].values )
    df_trans['sex']          = le.fit_transform( df_trans[['sex']].values )
    df_trans['ascites']      = le.fit_transform( df_trans[['ascites']].values )
    df_trans['hepatomegaly'] = le.fit_transform( df_trans[['hepatomegaly']].values )
    df_trans['spiders']      = le.fit_transform( df_trans[['spiders']].values )
    df_trans['edema']        = le.fit_transform( df_trans[['edema']].values )
    
    # NUMERICAL VARIABLES
    # STANDARD SCALER

    #df_trans['n_days']    = ss.fit_transform( df_trans[['n_days']].values )
    #df_trans['age']       = ss.fit_transform( df_trans[['age']].values )
    #df_trans['albumin']   = ss.fit_transform( df_trans[['albumin']].values )
    #df_trans['platelets'] = ss.fit_transform( df_trans[['platelets']].values )

    # ROBUST SCALER
    
    #df_trans['bilirubin']     = rr.fit_transform( df_trans[['bilirubin']].values )
    #df_trans['cholesterol']   = rr.fit_transform( df_trans[['cholesterol']].values )
    #df_trans['copper']        = rr.fit_transform( df_trans[['copper']].values )
    #df_trans['sgot']          = rr.fit_transform( df_trans[['sgot']].values )
    #df_trans['tryglicerides'] = rr.fit_transform( df_trans[['tryglicerides']].values )
    #df_trans['platelets']     = rr.fit_transform( df_trans[['platelets']].values )
    #df_trans['prothrombin']   = rr.fit_transform( df_trans[['prothrombin']].values )
    
    #df_trans = df_trans[['bilirubin','n_days','copper', 'hepatomegaly','prothrombin']]
    
    return(df_trans)

def jupyter_settings():
    %matplotlib inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [18, 9]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
   
    sns.set()
jupyter_settings()

In [ ]:
pd.set_option('display.float_format', lambda x: '%.6f' %x)

## 1 - DATAS

In [ ]:
# Train Datas - Windows
df_raw = pd.read_csv('../data/train.csv')

In [ ]:
## Renomeando as colunas, removendo letras maiusculas e substituindo espacos por _
cols_old = ['id', 'N_Days', 'Drug', 'Age', 'Sex', 'Ascites', 'Hepatomegaly','Spiders', 'Edema', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper',
            'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin','Stage', 'Status']

snakecase = lambda x: inflection.underscore(x.replace(' ', '_'))
cols_new = list( map( snakecase, cols_old ) )

df_raw.columns = cols_new

### 1.1 - DATASET SPLIT

In [ ]:
# Split dataset into train, test and validation datasets
X = df_raw.drop( ['status'], axis=1 )
y = df_raw['status'].copy()
x_train, x_valid, y_train, y_valid = ms.train_test_split( X, y, test_size=0.20, random_state=42 )
x_train, x_test, y_train, y_test   = ms.train_test_split( x_train, y_train, test_size=0.25, shuffle=True, random_state=42 )

print(f" x_train: {x_train.shape}/n y_train: {y_train.shape}/n x_valid: {x_valid.shape}/n y_valid: {y_valid.shape}/n x_test: {x_test.shape}/n y_test: {y_test.shape}/n")

In [ ]:
# Split dataset into train and validation datasets
X = df_raw.drop( ['status'], axis=1 )
y = df_raw['status'].copy()
x_train, x_valid, y_train, y_valid = ms.train_test_split( X, y, test_size=0.20, random_state=42 )

print(f"x_train: {x_train.shape}/n y_train: {y_train.shape}/n x_valid: {x_valid.shape}/n y_valid: {y_valid.shape}/n")

### 1.2 - DATA DESCRIPTION

In [ ]:
df1 = df_raw.copy()

In [ ]:
#prof = ProfileReport(df1)
#rof.to_file(output_file='data_descriptive.html')

In [ ]:
# Data Dimensions
print( 'Number of Rows: {}'.format( df1.shape[0] ) )
print( 'Number of Cols: {}'.format( df1.shape[1] ) )

In [ ]:
# Data Types
df1.dtypes

In [ ]:
# Check NAN Values
df1.isna().sum()

### 1.3 - DATA DESCRIPTIVE

In [ ]:
df1.info()

In [ ]:
df1.describe().T

In [ ]:
# Select columns where the content is different 0 or 1.
# numerical features
num_attributes = df1.select_dtypes(include=['int64', 'float64'])

# categorical features
cat_attributes = df1.select_dtypes(exclude=['int64', 'float64'])

In [ ]:
num_attributes.hist(bins=25);
plt.tight_layout()

In [ ]:
# Central Tendency - mean, median
ct1 = pd.DataFrame( num_attributes.apply( np.mean ) ).T
ct2 = pd.DataFrame( num_attributes.apply( np.median ) ).T

# Dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame( num_attributes.apply( np.std ) ).T
d2 = pd.DataFrame( num_attributes.apply( min ) ).T
d3 = pd.DataFrame( num_attributes.apply( max ) ).T
d4 = pd.DataFrame( num_attributes.apply( lambda x: x.max() - x.min() ) ).T
d5 = pd.DataFrame( num_attributes.apply( lambda x: x.skew() ) ).T
d6 = pd.DataFrame( num_attributes.apply( lambda x: x.kurtosis() ) ).T

# Concatenar
m = pd.concat( [ d2, d3, d4, ct1, ct2, d1, d5, d6 ] ).T.reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
m

# 2 - FEATURE ENGINEERING

In [ ]:
df2 = df1.copy()

# 3 - DATA FILTERING

In [ ]:
df3 = df2.copy()

# 4 - EXPLORATORY DATA ANALYSIS (EDA)

In [ ]:
df4 = df3.copy()

### 4.1 - UNIVARIATE ANALYSIS

In [ ]:
# DRUG
aux = df1[['id','drug','status']].groupby(['status','drug'] ).count().reset_index()
aux

In [ ]:
# SEX
aux_one = df1[['id','sex','status']].groupby(['status','sex'] ).count().reset_index()
aux_one

In [ ]:
# ASCITES
aux_two = df1[['id','ascites','status']].groupby(['status','ascites'] ).count().reset_index()
aux_two

In [ ]:
# HEPATOMEGALY
aux_three = df1[['id','hepatomegaly','status']].groupby(['status','hepatomegaly'] ).count().reset_index()
aux_three

In [ ]:
# SPIDERS
aux_four = df1[['id','spiders','status']].groupby(['status','spiders'] ).count().reset_index()
aux_four

In [ ]:
# EDEMA
aux_five = df1[['id','edema','status']].groupby(['status','edema'] ).count().reset_index()
aux_five

### 4.2 - BIVARIATE ANALYSIS

In [ ]:
#1.The interest on purchase the vehicle insurance is greater for customers that damaged their vehicle before and doesn't have insurance. (FALSE)
d1 = df4[( df4['previously_insured'] == 0) & (df4['vehicle_damage'] == 1)]

ax1 = d1[['response','id']].groupby('response').count().reset_index()
ax1['percentage'] = round(ax1['id'] / d1['id'].count()*100)
ax1['response'] = ax1['response'].apply(lambda x: 'No' if (x == 0) else 'Yes')

# Bar chart
fig1 = px.bar(ax1, x='response', 
              y='percentage', 
              color='response', 
              text='percentage', 
              width=1500,
              height=650,
              color_discrete_sequence=px.colors.qualitative.Pastel2)
fig1.update_traces(texttemplate='%{text:.2s}', 
                   textposition='outside')
fig1.update_layout(title_text="Hypothesis 1: Purchase Interest vs Vehicle Damage.", 
                   title_x=0.25,
                   font=dict(size=20))
fig1.show()

Hiphotesys is false once 75% of customers doesn't show interest in buying the vehicle insurance. 

In [ ]:
# 2. The interest on purchase the vehicle insurance is greater for woman than men. (FALSE)
ax2 = pd.crosstab(df4['gender'], df4['response'])
ax2['percentage'] = round(ax2[1]/(ax2[0]+ax2[1])*100)

# Creating new dataframe for a bar chart
aux2 = [['Female', 10],['Male',14]]
aux2_df = pd.DataFrame(aux2, columns=['gender','percentage'])

# Bar chart
fig = px.bar(aux2_df, 
             x='gender', 
             y='percentage', 
             color='gender', 
             text='percentage',
             width=1500,
             height=650, 
             color_discrete_sequence=px.colors.qualitative.Pastel2)
fig.update_traces(texttemplate='%{text:.2s}', 
                  textposition='outside')
fig.update_layout(title_text="Hypothesis 2: Purchase Interest vs Gender.", 
                   title_x=0.25,
                   font=dict(size=20))
fig.show()


Hiphotesys is false once 14% of the men show interest in buying the vehicle insurance against 10% of women. 

Hiphotesys is false, according the analysis, the period that the customers are with the company doesn't affect the interest in buying vehicle insurance. 

In [ ]:
# 3. The interest on purchase the vehicle insurance is greater for customers that spend less than 30k for annual premium.
aux5 = df4[df4['annual_premium'] > 30000][['id','response']]
aux6 = df4[df4['annual_premium'] <= 30000][['id','response']]

# Percentage of interested customers that spend more than 30k in annual premium
aux16 = aux5[['response','id']].groupby('response').count().reset_index()
aux16['percentage'] = round( aux16['id'] / aux5.shape[0] * 100 )

# Percentage of interested customers that spend less than 30k in annual premium
aux17 = aux6[['response','id']].groupby('response').count().reset_index()
aux17['percentage'] = round( aux17['id'] / aux6.shape[0] * 100 )

# Creating new dataframe for a bar chart
ax4 = [['30k or more in annual premium', 13],['30k or less in annual premium', 11]]
ax4_df = pd.DataFrame(ax4, columns=['annual_premium','percentage'])

# Bar chart
fig = px.bar(ax4_df,
             x='annual_premium',
             y='percentage', 
             color='annual_premium', 
             text='percentage', 
             width=1500, 
             height=650, 
             color_discrete_sequence=px.colors.qualitative.Pastel2)
fig.update_traces(texttemplate='%{text:.2s}',
                  textposition='outside')
fig.update_layout(title_text="Hypothesis 3: Purchase Interest vs Annual Premium",
                  title_x=0.17,
                  font=dict(size=20))
fig.show()

False, customers that spend more than 30k yearly show greter interest on purchase vehicle insurance.

In [ ]:
# 4. The interest on purchase the vehicle insurance is greater for customers that have driver license.
aux9 = pd.crosstab(df4['driving_license'], df4['response'])
aux9['percentage'] = round(aux9[1]/(aux9[0]+aux9[1])*100)

# Creating new dataframe for a bar chart
ax9 = [['No', 5 ],['Yes', 12]]
ax9_df = pd.DataFrame(ax9, columns=['driving_license','percentage'])

# Bar chart
fig = px.bar(ax9_df, 
             x='driving_license', 
             y='percentage', 
             color='driving_license', 
             text='percentage', 
             width=1500,
             height=650, 
             color_discrete_sequence=px.colors.qualitative.Pastel2 )
fig.update_traces(texttemplate='%{text:.2s}', 
                  textposition='outside')
fig.update_layout(title_text="Hypothesis 4: Purchase Interest vs Driving License.", 
                  title_x=0.25,
                  font=dict(size=20))
fig.show()

In [ ]:
# 5. The interest on purchase the vehicle insurance is greater for customers that have new cars.
aux10 = pd.crosstab(df4['vehicle_age'], df4['response'])
aux10['percentage'] = round(aux10[1]/(aux10[0]+aux10[1])*100)

# Creating new dataframe for a bar chart
ax10 = [['New', 4 ],['Used', 17],['Old', 29]]
ax10_df = pd.DataFrame(ax10, columns=['vehicle_age','percentage'])

# Bar chart
fig = px.bar(ax10_df, 
             x='vehicle_age', 
             y='percentage', 
             color='vehicle_age', 
             text='percentage',
             width=1500,
             height=650,
             color_discrete_sequence=px.colors.qualitative.Pastel2)
fig.update_traces(texttemplate='%{text:.2s}', 
                  textposition='outside')
fig.update_layout(title_text="Hypothesis 5: Purchase Interest vs Vehicle Age.", 
                   title_x=0.25,
                   font=dict(size=20))
fig.show()

In [ ]:
# 6. The interest on purchase the vehicle insurance is greater for customers that have new cars and have damaged their vehicles.
aux11 = df4[(df4['vehicle_damage'] == 1 )]
aux12 = pd.crosstab(aux11['vehicle_age'], aux11['response'])
aux12['percentage'] = round(aux12[1]/(aux12[0]+aux12[1])*100)

# Creating new dataframe for a bar chart
ax11 = [['New', 14 ],['Used', 27],['Old', 29]]
ax11_df = pd.DataFrame(ax11, columns=['vehicle_age','percentage'])

# Bar chart
fig = px.bar(ax11_df, 
             x='vehicle_age', 
             y='percentage', 
             color='vehicle_age', 
             text='percentage',
             width=1500,
             height=650, 
             color_discrete_sequence=px.colors.qualitative.Pastel2)
fig.update_traces(texttemplate='%{text:.2s}', 
                  textposition='outside')
fig.update_layout(title_text="Hypothesis 6: Purchase Interest vs New (damaged) cars.", 
                   title_x=0.25,
                   font=dict(size=20))
fig.show()

In [ ]:
# 7. The interest on purchase the vehicle insurance is lower for customers that are already insured.
aux15 = pd.crosstab(df4['previously_insured'], df4['response'])
aux15['percentage'] = round(aux15[1]/(aux15[0]+aux15[1])*100)
aux15

# Creating new dataframe for a bar chart
ax14 = [['No', 23 ],['Yes', 0]] 
ax14_df = pd.DataFrame(ax14, columns=['previously_insured','percentage'])

# Bar chart
fig = px.bar(ax14_df, 
             x='previously_insured', 
             y='percentage', 
             color='previously_insured', 
             text='percentage',
             width = 1500,
             height=650, 
             color_discrete_sequence=px.colors.qualitative.Pastel2)
fig.update_traces(texttemplate='%{text:.2s}', 
                  textposition='outside')
fig.update_layout(title_text="Hypothesis 7: Purchase Interest vs Previusly Insured.", 
                   title_x=0.25,
                   font=dict(size=20))
fig.show()


**Hypothesis Validation**

1. The interest on purchase the vehicle insurance is greater for customers that damaged their vehicle before and doesn't have insurance.
    **False, of the customers that damaged their car and doesn't have insurance, only 25% show interest in acquire vehicle insurance.**

2. The interest on purchase the vehicle insurance is greater for woman than men.
    **False, only 10% of women show interest in acquire vehicle insurance, whereas 13% of the men show interest on acquire vehicle insurance.**

3. The interest on purchase vehicle insurance is greater for vintage customers ( 7 months or more ).
    **False, the period that customers are on the company doensn't show influency on interest in buying vehicle insurance.**

4. The interest on purchase the vehicle insurance is greater for young customers.(Between 18 and 30 years old.)
    **False, customers that spend more than 30k yearly show greter interest on purchase vehicle insurance.**

5. The interest on purchase the vehicle insurance is greater for young customers.(Between 18 and 30 years old.)
    **False, adults and elderlies show greater interest on buying vehicle insurance.**

6. The interest on purchase the vehicle insurance is greater for customers that have driver license.
    **True, arround 12% of customers that hold a driving license show interest in buying the vehicle insurance.**

7. The interest on purchase the vehicle insurance is greater for customers that have new cars.
    **False, the interest is greater for customers that own an old car.**

8. The interest on purchase the vehicle insurance is greater for customers that have new cars and have damaged their vehicles.
    **False, of the customers who damaged their car, the ones that own a old car show greater interest in buying the vehicle insurance (29%), followed by customers that own used cars (27%).**

9. The interest on purchase the vehicle insurance is greater for elderly women.
    **False, adult women show greater interest in buying the vehicle insurance.**

10. The interest on purchase the vehicle insurance is lower for customers that are already insured.
    **True, less than 1% of customers already insured show interest on purchase the vehicle insurance.**


### 4.3 - MULTIVARIATE ANALYSIS

In [ ]:
correlation = df4.corr().round(2)
plt.figure(figsize = (14,7))
sns.heatmap(correlation, annot = True)

# 5 - DATA PREPARATION

n_days - standard_scaler
age    - parece que esta em dias, posso transformar em anos, standard_scaler
bilirubin - tem outliers, que parecem parte do processo. Aplicar robust_scaller ou filtrar valore e usar apenas menores que 10.
cholesterol - tem outliers, que parecem parte do processo. Aplicar robust_scaller
albumin - standard_scaler
copper - Aplicar robust_scaller
alk_phos - Aplicar robust_scaller
sgot - Aplicar robust_scaller
tryglicerides - Aplicar robust_scaller
platelets - standard_scaler
prothrombin - Aplicar robust_scaller

In [ ]:
df5 = df4.copy()

In [ ]:
# ENCODER
le = pp.LabelEncoder()

df5['drug'] = le.fit_transform( df5[['drug']].values )
df5['sex'] = le.fit_transform( df5[['sex']].values )
df5['ascites'] = le.fit_transform( df5[['ascites']].values )
df5['hepatomegaly'] = le.fit_transform( df5[['hepatomegaly']].values )
df5['spiders'] = le.fit_transform( df5[['spiders']].values )
df5['edema'] = le.fit_transform( df5[['edema']].values )

In [ ]:
# STANDARD SCALER
ss = pp.StandardScaler()

df5['n_days'] = ss.fit_transform( df5[['n_days']].values )
df5['age']    = ss.fit_transform( df5[['age']].values )
df5['albumin'] = ss.fit_transform( df5[['albumin']].values )
df5['platelets'] = ss.fit_transform( df5[['platelets']].values )


# ROBUST SCALER
rr = pp.RobustScaler()
df5['bilirubin'] = rr.fit_transform( df5[['bilirubin']].values )
df5['cholesterol'] = rr.fit_transform( df5[['cholesterol']].values )
df5['copper'] = rr.fit_transform( df5[['copper']].values )
df5['sgot'] = rr.fit_transform( df5[['sgot']].values )
df5['tryglicerides'] = rr.fit_transform( df5[['tryglicerides']].values )
df5['platelets'] = rr.fit_transform( df5[['platelets']].values )
df5['prothrombin'] = rr.fit_transform( df5[['prothrombin']].values )

In [ ]:
x_training   = data_trans(x_train)
x_validation = data_trans(x_valid)
x_testing    = data_trans(x_test)

# 6 - FEATURE SELECTION

In [ ]:
df6 = x_train.copy()

In [ ]:
# model definition
forest = en.ExtraTreesClassifier( n_estimators=500, random_state=0, n_jobs=-1 )

# data preparation
df6_n = df6
y_train_n  = y_train.values
forest.fit( df6_n, y_train_n )

In [ ]:
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0 )
indices = np.argsort(importances)[::-1]

# Print the feature ranking 
print("Feature ranking:")
df = pd.DataFrame()
for i,j in zip( df6_n, forest.feature_importances_ ):
    aux = pd.DataFrame( {'feature': i, 'importance': j}, index=[0] )
    df = pd.concat( [df,aux], axis = 0 )
    
print( df.sort_values( 'importance', ascending=False ) )

# Plot the impurity-based feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(df6_n.shape[1]), importances[indices], color="r", yerr=std[indices], align="center")
plt.xticks(range(df6_n.shape[1]), indices)
plt.xlim([-1, df6_n.shape[1]])
plt.show()

# 7 - MACHINE LEARNING MODELS

## 7.1 - Logistic Regression

In [ ]:
# Model Definition
clf_lr = LogisticRegression()

# Model Training
clf_lr.fit(x_training, y_train)

# Model Prediction
yhat_lr = clf_lr.predict_proba(x_validation)

In [ ]:
# Evaluation
lr_logloss = mt.log_loss(y_valid, yhat_lr )
lr_logloss

## 7.2 - LGBM

In [ ]:
# Model Definition
clf_lgbm = lgbm.LGBMClassifier()

# Model Training
clf_lgbm.fit(x_training, y_train)

# Model Prediction
yhat_lgbm = clf_lgbm.predict_proba(x_validation)

In [ ]:
# Evaluation
lgbm_logloss = mt.log_loss(y_valid, yhat_lgbm )
lgbm_logloss

# 7.3 - CatBoost

In [ ]:
cb_train = data_trans(X)
cby_train = y

In [ ]:
# Definition
cb_model = CatBoostClassifier()

# Model training
cb_model.fit(cb_train, cby_train)

# Model Prediction
#yhat_cb = cb_model.predict_proba(x_validation)

In [ ]:
# Evaluation
cb_logloss = mt.log_loss(y_valid, yhat_cb)
cb_logloss

## 9 - HYPERPARAMETER FINE TUNING

# 10 - SUBMISSIONS

In [ ]:
df_test = pd.read_csv('../data/test.csv')

In [ ]:
## Renomeando as colunas, removendo letras maiusculas e substituindo espacos por _
cols_old = ['id', 'N_Days', 'Drug', 'Age', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides',\
            'Platelets', 'Prothrombin', 'Stage']

snakecase = lambda x: inflection.underscore(x.replace(' ', '_'))
cols_new = list( map( snakecase, cols_old ) )

df_test.columns = cols_new

In [ ]:
df_pred = data_trans(df_test)

In [ ]:
# Model Prediction
yhat_submission = cb_model.predict_proba(df_pred)

In [ ]:
sub_mission = df_test.copy()
sub_mission['Status_C'] = yhat_submission[:, 0]
sub_mission['Status_CL'] = yhat_submission[:, 1]
sub_mission['Status_D'] = yhat_submission[:, 2]

In [ ]:
submission = sub_mission[['id','Status_C','Status_CL','Status_D']]

In [ ]:
submission.to_csv("submission_three.csv", index=None, sep=',')